<a href="https://colab.research.google.com/github/cyingliu/Text-Enhanced-MedCLIP/blob/main/testing_clip_medqa_pytorch_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
! pip install datasets transformers
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 29.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
ERROR: Operation cancelled by user


In [1]:
import transformers

print(transformers.__version__)

4.40.2


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = 100
print('using device:', device)

using device: cpu


In [3]:
model_checkpoint = "openai/clip-vit-base-patch32"

In [4]:
from datasets import load_dataset, load_metric
datasets = load_dataset("bigbio/med_qa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for bigbio/med_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/med_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.war

Generating train split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/dc86b4af1d384a2680873fd3704ef3e09f2446bcfbf86dcc7655b8be63682696/data_clean/questions/US/train.jsonl


Generating test split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/dc86b4af1d384a2680873fd3704ef3e09f2446bcfbf86dcc7655b8be63682696/data_clean/questions/US/test.jsonl


Generating validation split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/dc86b4af1d384a2680873fd3704ef3e09f2446bcfbf86dcc7655b8be63682696/data_clean/questions/US/dev.jsonl


In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(datasets["train"])

,meta_info,question,answer_idx,answer,options
0,step2&3,"A 50-year-old woman, gravida 5, para 5, comes to the physician for the evaluation of decreased sexual desire for approximately 6 months. She has been sexually active with her husband but reports that she has no desire in having sexual intercourse anymore. She states that she feels guilty and is worried about losing her husband if this problem goes on for a longer period of time. She also reports that they have had several fights recently due to financial problems. She has problems going to sleep and wakes up often, and is tired throughout the day. One year ago, the patient underwent hysterectomy with bilateral salpingo-oophorectomy due to uterine prolapse. Her last menstrual period was 2 years ago. She does not smoke. She drinks 3–4 glasses of wine daily. Vital signs are within normal limits. Physical examination shows no abnormalities except for an enlarged liver. Which of the following most likely explains this patient's loss of libido?",C,Decreased testosterone,"[{'key': 'A', 'value': 'Chronic alcohol intake'}, {'key': 'B', 'value': 'Major depressive disorder'}, {'key': 'C', 'value': 'Decreased testosterone'}, {'key': 'D', 'value': 'Elevated prolactin'}, {'key': 'E', 'value': 'Stress'}]"
1,step1,"A 35-year-old man comes to the physician because of worsening pain in his lower back, knees, and shoulders over the past few years. He used to be able to touch his fingers to his toes while standing; now he has difficulty touching his shins. He is wearing a shirt with dark brown stains around the armpits. Physical examination shows bluish-brown sclerae and thickening of the external ear. The range of motion of the affected joints is decreased. X-rays of the spine show calcification of multiple lumbar intervertebral discs. The patient's condition is most likely caused by impaired metabolism of which of the following?",D,Tyrosine,"[{'key': 'A', 'value': 'Homocysteine'}, {'key': 'B', 'value': 'Tryptophan'}, {'key': 'C', 'value': 'Hypoxanthine'}, {'key': 'D', 'value': 'Tyrosine'}, {'key': 'E', 'value': 'Ornithine'}]"
2,step2&3,"A 33-year-old man is brought into the emergency department with fever, lethargy, and confusion. He is a cachectic man in acute distress, unable to respond to questions or follow commands. His friend confides that the patient has been sexually active with multiple male partners and was diagnosed with HIV several months ago, but was lost to follow up. Based on prior records, his most recent CD4 count was 65 cells/uL. Which of the following is the most appropriate next step in management?",E,Neurological exam with fundoscopy,"[{'key': 'A', 'value': 'Recheck CD4 and HIV viral load serologies'}, {'key': 'B', 'value': 'Lumbar puncture'}, {'key': 'C', 'value': 'MRI brain with contrast'}, {'key': 'D', 'value': 'CT head without contrast'}, {'key': 'E', 'value': 'Neurological exam with fundoscopy'}]"
3,step1,An investigator is studying brachial artery reactivity in women with suspected coronary heart disease. The brachial artery diameter is measured via ultrasound before and after intra-arterial injection of acetylcholine. An increase of 7% in the vascular diameter is noted. The release of which of the following is most likely responsible for the observed effect?,A,Nitric oxide from endothelial cells,"[{'key': 'A', 'value': 'Nitric oxide from endothelial cells'}, {'key': 'B', 'value': 'Endothelin from the peripheral vasculature'}, {'key': 'C', 'value': 'Serotonin from neuroendocrine cells'}, {'key': 'D', 'value': 'Norepinephrine from the adrenal medulla'}, {'key': 'E', 'value': 'Atrial natriuretic peptide from atrial myocytes'}]"
4,step1,"A 59-year-old man with a history of congestive heart failure presents to his cardiologist for a follow-up visit. His past medical history is notable for diabetes mellitus, hypertension, and obesity. He takes metformin, glyburide, aspirin, lisinopril, and metoprolol. He has a 40 pack-year smoking history and drinks alcohol socially.

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [9]:
tokenizer

CLIPTokenizerFast(name_or_path='openai/clip-vit-base-patch32', vocab_size=49408, model_max_length=77, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	49406: AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	49407: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
answer_indices = ["A", "B", "C", "D", "E"]

def getValue(options, label):
  return list(filter(lambda option : option['key'] == label, options))[0]['value']

def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 5 for context in examples["question"]]
    second_sentences = [[getValue(examples['options'][i], answer_idx) for answer_idx in answer_indices] for i, _ in enumerate(examples["question"])]

    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+5] for i in range(0, len(v), 5)] for k, v in tokenized_examples.items()}

In [13]:
examples = datasets["train"][:2]
features = preprocess_function(examples)
print(features)

{'input_ids': [[[49406, 320, 273, 274, 268, 935, 268, 896, 11195, 2308, 536, 273, 273, 2898, 619, 2631, 4215, 593, 8405, 5067, 565, 4706, 269, 1043, 4218, 585, 2760, 272, 575, 1468, 537, 791, 1025, 46344, 519, 5325, 5778, 750, 1573, 537, 2019, 23824, 18962, 269, 1043, 12376, 4808, 1123, 537, 533, 6499, 638, 320, 5547, 556, 899, 12769, 269, 899, 9543, 533, 280, 278, 269, 278, 6858, 325, 263, 274, 277, 269, 49407, 49407, 548, 901, 35868, 49407], [49406, 320, 273, 274, 268, 935, 268, 896, 11195, 2308, 536, 273, 273, 2898, 619, 2631, 4215, 593, 8405, 5067, 565, 4706, 269, 1043, 4218, 585, 2760, 272, 575, 1468, 537, 791, 1025, 46344, 519, 5325, 5778, 750, 1573, 537, 2019, 23824, 18962, 269, 1043, 12376, 4808, 1123, 537, 533, 6499, 638, 320, 5547, 556, 899, 12769, 269, 899, 9543, 533, 280, 278, 269, 278, 6858, 325, 263, 274, 49407, 49407, 685, 69, 924, 3165, 637, 49407], [49406, 320, 273, 274, 268, 935, 268, 896, 11195, 2308, 536, 273, 273, 2898, 619, 2631, 4215, 593, 8405, 5067, 565, 4706, 

In [12]:
idx = 1
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(5)]

['<|startoftext|>a 3 - month - old baby died suddenly at night while asleep. his mother noticed that he had died only after she awoke in the morning. no cause of death was determined based on the autopsy. which of the following precautions could have prevented the death of the baby? <|endoftext|><|endoftext|>placing the infant in a supine position on a firm mattress while sleeping <|endoftext|>',
 '<|startoftext|>a 3 - month - old baby died suddenly at night while asleep. his mother noticed that he had died only after she awoke in the morning. no cause of death was determined based on the autopsy. which of the following precautions could have prevented the death of the baby? <|endoftext|><|endoftext|>routine postnatal electrocardiogram ( ecg ) <|endoftext|>',
 '<|startoftext|>a 3 - month - old baby died suddenly at night while asleep. his mother noticed that he had died only after she awoke in the morning. no cause of death was determined based on the autopsy. which of the following pr

In [14]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

Map:   0%|          | 0/1273 [00:00<?, ? examples/s]

Map:   0%|          | 0/1272 [00:00<?, ? examples/s]

In [16]:
encoded_datasets = encoded_datasets.rename_column("answer_idx", "label")

In [17]:
# Define a function to convert labels
def convert_labels(example):
    label_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
    example['label'] = label_mapping[example['label']]
    return example

# Apply the function to the 'label' column
encoded_datasets = encoded_datasets.map(convert_labels)

Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

Map:   0%|          | 0/1273 [00:00<?, ? examples/s]

Map:   0%|          | 0/1272 [00:00<?, ? examples/s]

In [18]:
encoded_datasets

DatasetDict({
    train: Dataset({
        features: ['meta_info', 'question', 'label', 'answer', 'options', 'input_ids', 'attention_mask'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['meta_info', 'question', 'label', 'answer', 'options', 'input_ids', 'attention_mask'],
        num_rows: 1273
    })
    validation: Dataset({
        features: ['meta_info', 'question', 'label', 'answer', 'options', 'input_ids', 'attention_mask'],
        num_rows: 1272
    })
})

In [19]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        # print(batch_size)
        num_choices = len(features[0]["input_ids"])
        # print(num_choices)
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        # print(len(flattened_features))

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        # print(batch['input_ids'].shape)

        # Un-flatten
        # batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        # print(batch['input_ids'].shape)
        return batch

In [20]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [21]:
print(batch['input_ids'].shape)

torch.Size([50, 77])


In [23]:
[tokenizer.decode(batch["input_ids"][i]) for i in range(10)]

['<|startoftext|>a 2 3 - year - old pregnant woman at 2 2 weeks gestation presents with burning upon urination. she states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. she otherwise feels well and is followed by a doctor for her pregnancy. her temperature is 9 7. 7 ° f ( 3 6. <|endoftext|><|endoftext|>ampicillin <|endoftext|>',
 '<|startoftext|>a 2 3 - year - old pregnant woman at 2 2 weeks gestation presents with burning upon urination. she states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. she otherwise feels well and is followed by a doctor for her pregnancy. her temperature is 9 7. 7 ° f ( 3 <|endoftext|><|endoftext|>ceftriaxone <|endoftext|>',
 '<|startoftext|>a 2 3 - year - old pregnant woman at 2 2 weeks gestation presents with burning upon urination. she states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. she

In [24]:
encoded_datasets

DatasetDict({
    train: Dataset({
        features: ['meta_info', 'question', 'label', 'answer', 'options', 'input_ids', 'attention_mask'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['meta_info', 'question', 'label', 'answer', 'options', 'input_ids', 'attention_mask'],
        num_rows: 1273
    })
    validation: Dataset({
        features: ['meta_info', 'question', 'label', 'answer', 'options', 'input_ids', 'attention_mask'],
        num_rows: 1272
    })
})

In [26]:
batch_size=32

In [27]:
from torch.utils.data import DataLoader

data_collator = DataCollatorForMultipleChoice(tokenizer)

# Set the format to PyTorch tensors
encoded_datasets.set_format(type='torch', columns=['label', 'input_ids', 'attention_mask'])

# Create a DataLoader
loader_train = DataLoader(encoded_datasets['train'], batch_size=batch_size, collate_fn=data_collator)

In [28]:
loader_val = DataLoader(encoded_datasets['validation'], batch_size=batch_size, collate_fn=data_collator)

In [ ]:
loader_test = DataLoader(encoded_datasets['test'], batch_size=batch_size, collate_fn=data_collator)

In [34]:
for t, batch in enumerate(loader_val):
    print(batch)
    break

{'input_ids': tensor([[49406,   320,   273,  ...,   535,  2985, 49407],
        [49406,   320,   273,  ..., 25824,   530, 49407],
        [49406,   320,   273,  ...,   546,   530, 49407],
        ...,
        [49406,   320,   274,  ...,   751,  5230, 49407],
        [49406,   320,   274,  ...,   751,  5230, 49407],
        [49406,   320,   274,  ...,   539,  3964, 49407]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([3, 0, 4, 2, 0, 2, 4, 2, 0, 0, 0, 3, 1, 0, 1, 4, 2, 0, 1, 1, 3, 3, 0, 3,
        2, 3, 0, 3, 4, 3, 4, 0])}


In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'cs231n/project/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/cs231n/project


In [35]:
from transformers import AutoModel

import torch.nn as nn

class CLIPTextFinetunedModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.text_model = AutoModel.from_pretrained(model_checkpoint).text_model.to(device)
        self.classifier = nn.Linear(512, 1).to(device)

    def forward(self, input_ids, attention_mask):
        num_choices = 5
        count = input_ids.shape[0] // num_choices # 160 / 32 = 5
        text_model_output = self.text_model(input_ids, attention_mask)
        reshaped = text_model_output.pooler_output.view(count, num_choices, 512).to(device) # (160, 512) -> (32, 5, 512)
        logits = self.classifier(reshaped).to(device) # (32, 5, 1)
        return logits

In [36]:
model = CLIPTextFinetunedModule()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
model

MyModule(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05, 

In [ ]:
import wandb

# 🐝 initialise a wandb run
wandb.init(
    project="clip_medqa_pytorch_finetune",
    config={
        "epochs": 10,
        "batch_size": 32,
        "lr": 5e-5,
        })

train/example_ct,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
train/per_batch_train_loss,███████████████▅▆▇▁▄▃▅▅▅▃▁▃▃▂▃▃▂▂▃▂▁▁▃▁▂
train/train_accuracy,▁▁▂▄▆▇▇███
train/train_loss,███▅▃▂▂▁▁▁
val/granular_val_accuracy,▄▄▇▄▄▃▃▅▆▃▆▅▇█▅▂▇▅▃▂▆▅▅▆▆▆▇▆▇█▆▇█▆▇▆▅▆▆▁
val/granular_val_loss,▅▅▅▅▅▅▅▅▅▅▄▄▃▂▄▄▄▅▆▆▄▃▄▃▄▅▄▄▂▂▅▁▃▄▄▃▄▃▄█
val/val_accuracy,▆▃▃▁▅▇▇█▂▃
val/val_loss,▁▁▁▄▄▅▅▆▇█
train/example_ct,102068
train/per_batch_train_loss,1.25702
train/train_accuracy,0.61034


In [ ]:
from torch.utils.data import DataLoader

data_collator = DataCollatorForMultipleChoice(tokenizer)

# Set the format to PyTorch tensors
encoded_datasets.set_format(type='torch', columns=['label', 'input_ids', 'attention_mask'])

# Create a DataLoader
# loader_train = DataLoader(encoded_datasets['train'].select(range(320)), batch_size=batch_size, collate_fn=data_collator)
# loader_val = DataLoader(encoded_datasets['validation'].select(range(320)), batch_size=batch_size, collate_fn=data_collator)
loader_train = DataLoader(encoded_datasets['train'], batch_size=batch_size, collate_fn=data_collator)
loader_val = DataLoader(encoded_datasets['validation'], batch_size=batch_size, collate_fn=data_collator)

In [ ]:
import torch.nn.functional as F

def validate_model(loader, model, epoch):
    # if loader.dataset.train:
    #     print('Checking accuracy on validation set')
    # else:
    #     print('Checking accuracy on test set')
    print('Checking accuracy')
    num_correct = 0
    num_samples = 0
    loss = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for batch in loader:
            # batch = batch.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            input_ids = batch['input_ids']
            input_ids = input_ids.to(device=device, dtype=torch.long)
            attention_mask = batch['attention_mask']
            attention_mask = attention_mask.to(device=device, dtype=torch.long)
            y = batch['labels']
            y = y.to(device=device, dtype=torch.long)
            logits = model(input_ids, attention_mask)
            probabilities = F.softmax(logits, dim=1).squeeze()
            loss += F.cross_entropy(probabilities, y) * y.size(0)
            # print(probabilities.shape)
            # print(y)
            _, preds = probabilities.max(1)
            # preds = preds.squeeze()
            # print(preds)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
            if epoch == False:
              break
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return loss / num_samples, acc

In [ ]:
validate_model(loader_val, model, epoch=False)

Checking accuracy
Got 4 / 32 correct (12.50)


(tensor(1.6132, device='cuda:0'), 0.125)

In [ ]:
import torch.nn.functional as F
import math
import torch

def train(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.

    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for

    Returns: Nothing, but prints model accuracies during training.
    """
    n_steps_per_epoch = math.ceil(len(loader_train) / batch_size)
    example_ct = 0
    step_ct = 0

    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for epoch in range(epochs):
        print('Epoch %d' % (epoch))
        print()
        for step, batch in enumerate(loader_train):
            model.train()  # put model to training mode
            # batch = batch.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            input_ids = batch['input_ids']
            input_ids = input_ids.to(device=device, dtype=torch.long)
            attention_mask = batch['attention_mask']
            attention_mask = attention_mask.to(device=device, dtype=torch.long)
            y = batch['labels']
            y = y.to(device=device, dtype=torch.long)

            logits = model(input_ids, attention_mask) # (32, 5, 1)
            probabilities = F.softmax(logits, dim=1).squeeze() # (32, 5)
            # TODO: check if F.cross_entropy does softmax.
            train_loss = F.cross_entropy(probabilities, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            train_loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            ### Wandb ###
            example_ct += y.size(0)
            metrics = {"train/per_batch_train_loss": train_loss,
                       "train/example_ct": example_ct}

            if step + 1 < n_steps_per_epoch:
                # 🐝 Log train metrics to wandb
                wandb.log(metrics)

            step_ct += 1

            if step % 100 == 0:
                print('Step %d' % (step))
                granular_val_loss, granular_val_accuracy = validate_model(loader_val, model, epoch=False)
                # 🐝 Log train and validation metrics to wandb
                granular_val_metrics = {"val/granular_val_loss": granular_val_loss,
                              "val/granular_val_accuracy": granular_val_accuracy,}
                wandb.log(granular_val_metrics)
                print(f"Valid Loss: {granular_val_loss:3f}, Accuracy: {granular_val_accuracy:.2f}")

        val_loss, val_accuracy = validate_model(loader_val, model, epoch=True)
        train_loss, train_accuracy = validate_model(loader_train, model, epoch=True)

        # 🐝 Log train and validation metrics to wandb
        val_metrics = {"val/val_loss": val_loss,
                      "val/val_accuracy": val_accuracy,
                       "train/train_loss": train_loss,
                       "train/train_accuracy": train_accuracy,}
        wandb.log({**metrics, **val_metrics})

        print(f"Train Loss: {train_loss:.3f}, Train Accuracy: {train_accuracy:.2f}, Valid Loss: {val_loss:3f}, Valid Accuracy: {val_accuracy:.2f}")

        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(), f"./clip_medqa_pytorch_finetune-epoch-{epoch+1}.pth")

In [ ]:
import torch.optim as optim

learning_rate = 5e-5
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-3)

In [ ]:
train(model, optimizer, epochs=100)

Epoch 0

Step 0
Checking accuracy
Got 8 / 32 correct (25.00)
Valid Loss: 1.608750, Accuracy: 0.25
Step 100
Checking accuracy
Got 6 / 32 correct (18.75)
Valid Loss: 1.608214, Accuracy: 0.19
Step 200
Checking accuracy
Got 9 / 32 correct (28.12)
Valid Loss: 1.608652, Accuracy: 0.28
Step 300
Checking accuracy
Got 5 / 32 correct (15.62)
Valid Loss: 1.609881, Accuracy: 0.16
Checking accuracy
Got 280 / 1272 correct (22.01)
Checking accuracy
Got 2092 / 10178 correct (20.55)
Train Loss: 1.609, Train Accuracy: 0.21, Valid Loss: 1.609163, Valid Accuracy: 0.22
Epoch 1

Step 0
Checking accuracy
Got 5 / 32 correct (15.62)
Valid Loss: 1.609449, Accuracy: 0.16
Step 100
Checking accuracy
Got 5 / 32 correct (15.62)
Valid Loss: 1.609593, Accuracy: 0.16
Step 200
Checking accuracy
Got 7 / 32 correct (21.88)
Valid Loss: 1.594354, Accuracy: 0.22
Step 300
Checking accuracy
Got 7 / 32 correct (21.88)
Valid Loss: 1.608641, Accuracy: 0.22
Checking accuracy
Got 270 / 1272 correct (21.23)
Checking accuracy
Got 253